In [3]:
%ls
%cd ..

analyze.ipynb  generate.py  old_analyze.py  __pycache__/
/data/joshua_clymer/owen-workspace/value_instability


In [4]:
import pandas as pd
scores_without_reasoning_path = 'llama3-instruct/scores_without_reasoning_harmless_standard.jsonl'
scores_with_reasoning_path = 'llama3-instruct/scores_with_reasoning_harmless_standard.jsonl'
scores_without_reasoning = pd.read_json(scores_without_reasoning_path, orient='records', lines=True)
scores_with_reasoning = pd.read_json(scores_with_reasoning_path, orient='records', lines=True)

In [5]:
# Print columns 
print(scores_without_reasoning.columns)
print(scores_with_reasoning.columns)

Index(['top_k_scores', 'top_k_ids'], dtype='object')
Index(['top_k_scores', 'top_k_ids'], dtype='object')
